# Analyzing profitable App profiles on Google Play Store and iOS App Store

We are trying to analyze apps data from Google Play Store and iOS App Store to help developers
understand profitable app profiles.
For a company building only free apps, the main source of revenue is customers. More the amount
of customers enagaging with the apps and ads, more the revenue.

### Exploring the datasets
As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:

A data set containing data about approximately ten thousand Android apps from Google Play. Available directly from [this link.](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv)
A data set containing data about approximately seven thousand iOS apps from the App Store. Available directly from [this link.](https://dq-content.s3.amazonaws.com/350/AppleStore.csv)

In [1]:
from csv import reader

In [57]:
# ios AppStore
ios_opened_file = open('AppleStore.csv')
ios_read_file = reader(ios_opened_file)
ios = list(ios_read_file)
ios_header = ios[0]
ios = ios[1:]

# Google PlayStore
android_opened_file = open('googleplaystore.csv')
android_read_file = reader(android_opened_file)
android = list(android_read_file)
android_header = android[0]
android = android[1:]

In [58]:
# helper function to print rows in a readable way
# dataset is expected to a list of lists
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows: ', len(dataset))
        print('Number of columns: ', len(dataset[0]))

In [59]:
print(android_header)
print('\n')
explore_data(android, 100, 110, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Natural recipes for your beauty', 'BEAUTY', '4.7', '1150', '9.8M', '100,000+', 'Free', '0', 'Everyone', 'Beauty', 'May 15, 2018', '4.0', '4.1 and up']


['BestCam Selfie-selfie, beauty camera, photo editor', 'BEAUTY', '3.9', '1739', '21M', '500,000+', 'Free', '0', 'Everyone', 'Beauty', 'July 12, 2018', '1.0.6', '4.0.3 and up']


['Mirror - Zoom & Exposure -', 'BEAUTY', '3.9', '32090', 'Varies with device', '1,000,000+', 'Free', '0', 'Everyone', 'Beauty', 'October 24, 2016', 'Varies with device', 'Varies with device']


['Beauty Selfie Camera', 'BEAUTY', '4.2', '2225', '52M', '500,000+', 'Free', '0', 'Everyone', 'Beauty', 'February 28, 2018', '1.6', '4.1 and up']


['Hairstyles step by step', 'BEAUTY', '4.6', '4369', '14M', '100,000+', 'Free', '0', 'Everyone', 'Beauty', 'July 25, 2018', '1.9', '4.0.3 and up']


['Filters for Selfie',

We can see that we have data of 10841 Android apps. Columns that might be interesting for our analysis are Category, Reviews, Type, Price, Genre, Installs and App

In [60]:
print(ios_header)
print('\n')
explore_data(ios, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows:  7197
Number of columns:  16


We can see that we have data of 7197 iOS apps. Columns that might be interesting for our analysis are price, currency, rating_count_tot, 
rating_count_ver, prime_genre and track_name. More details about the columns are available [here.](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)


### Looking for missing column enteries in rows

In [61]:
header_len = len(android_header)
for row in android:
    if len(row) != header_len:
        print(row)
        print(android.index(row))

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
10472


We see above that the row 10472 has a missing value for column 'Category' causing a column shift in all the values.
Here we can either delete the row using "del android[10472]" or we can insert the Category value if we can find out the value.
Quick search on PlayStore show us that this [app](https://play.google.com/store/apps/details?id=com.lifemade.internetPhotoframe)
belongs to the 'LifeStyle' category. So we can also insert that value into the row

In [62]:
android[10472].insert(1, "LIFESTYLE")

We did not find any such missing column value for the iOS dataset

### Looking for duplicate enteries

In [63]:
unique_apps = []
duplicate_apps = []

for row in android:
    if row[0] in unique_apps:
        duplicate_apps.append(row[0])
    else:
        unique_apps.append(row[0])

In [64]:
print(len(duplicate_apps))
print(len(unique_apps))

1181
9660


We can see that there are 1181 duplicate app enteries. Lets take a closer look at the duplicates

In [65]:
for row in android:
    if row[0] == 'Instagram':
        print(row)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


We see that the 4th column which is the count of reviews is different. This probably means that the different enteries were recorded
at different points of time. We choose to keep only the latest entry with the max count of reviews as that will give us a more 
reliable avg. rating of the app.

In [66]:
reviews_max = {}
for row in android:
    name = row[0]
    n_reviews = float(row[3])
    # if the app is not present, we add the key and review to the dictionary
    if name not in reviews_max:
        reviews_max[name] = n_reviews
        
    # if the app is present and review count in the dictionary is less that current 
    # app entry we replace the review count with the new higher review count
    elif name in reviews_max and n_reviews > reviews_max[name]:
        reviews_max[name] = n_reviews
    

Previously we found that there are 1,181 cases where an app occurs more than once, so the length of our dictionary (of unique apps) 
should be equal to the difference between the length of our data set and 1,181.

In [67]:
print('Expected Length ', len(android) - 1181)
print('Actual Length ', len(reviews_max))

Expected Length  9660
Actual Length  9660


In [68]:
android_clean = []
already_added = []
for row in android:
    name = row[0]
    n_reviews = float(row[3])
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(row)
        already_added.append(name)

print(len(android_clean))        

9660


In [69]:
unique_apps_ios = []
duplicate_apps_ios = []

for row in ios:
    if row[0] in unique_apps_ios:
        duplicate_apps_ios.append(row[0])
    else:
        unique_apps_ios.append(row[0])
        
print(len(duplicate_apps_ios))
print(len(unique_apps_ios))

0
7197


So there are no duplicate enteries in the iOS AppStore dataset

### Removing non-English app

We are focusing our analysis only towards apps in English language and need to remove other language apps.
All the characters commonly used in English text are in the ASCII range 0 to 127. So we detect app whose names 
have non-English characters (ASCII outside the range 0-127) and remove rows corresponding to those enteries.

In [70]:
# function to look for characters in string that are outside the range of English character ASCII values
def is_english_str(str):
    for char in str:
        if ord(char) > 127:
            return False
    return True    

In [71]:
print(is_english_str("爱奇艺PPS -《欢乐颂2》电视剧热播"))
print(is_english_str("Instagram"))

False
True


In [72]:
# this function has the problem that it also marks emojis and strings like Docs To Go™ Free Office Suite
# as non-English as '™' and emojis fall outside the valid English ASCII range.
print(is_english_str("Docs To Go™ Free Office Suite"))

False


In [73]:
# edit the function to look for more than 3 invlaid characters
def is_english_str(str):
    count_invalid = 0
    for char in str:
        if ord(char) > 127:
            count_invalid += 1
    if count_invalid > 3:
        return False
    return True    

print(is_english_str("Docs To Go™ Free Office Suite"))

True


In [74]:
android_english_count = 0
for row in android_clean:
    if is_english_str(row[0]):
        android_english_count += 1
print('Total Android apps: ', len(android_clean))         
print('English Android apps: ',android_english_count)        

Total Android apps:  9660
English Android apps:  9615


In [75]:
# remove non-English app rows from the android_clean dataset
android_english = []
for row in android_clean:
    if is_english_str(row[0]):
        android_english.append(row)

print('English Android apps: ', len(android_english))        

English Android apps:  9615


In [76]:
ios_english_count = 0
for row in ios:
    if is_english_str(row[0]):
        ios_english_count += 1
print('Total ios apps: ', len(ios))         
print('English ios apps: ',ios_english_count)   

# there are no iOS apps which need to be removed

Total ios apps:  7197
English ios apps:  7197


### Remove non-free apps from the dataset

We are only focussed towards free apps as revenue is generated by user engagement with the app and ads only.
We look for non-free apps in the dataset and remove them

In [77]:
explore_data(android_english, 0, 2)
explore_data(ios, 0, 2)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']




In [78]:
# We see that column 7 of the android dataset is used for price information
android_price_info_dict = {}
for row in android_english:
    price = row[6]
    if price in android_price_info_dict:
        android_price_info_dict[price] += 1
    else:
        android_price_info_dict[price] = 1
print(android_price_info_dict)        

ios_price_info_dict = {'Free':0, 'Paid':0}
for row in ios:
    if row[4] == '0.0':
        ios_price_info_dict['Free'] += 1
    else:
        ios_price_info_dict['Paid'] += 1
print(ios_price_info_dict)        
        

{'Free': 8864, 'Paid': 750, 'NaN': 1}
{'Free': 4056, 'Paid': 3141}


We can see that there for Android there are 8865 (including the NaN) free apps and 750 paid apps.
For iOS, there are 4065 free appps and 3141 paid apps.

In [79]:
# lets remove the non-free apps 
android_final = []
ios_final = []

for app in android_english:
    price = app[7]
    if price == '0':
        android_final.append(app)
        
for app in ios:
    price = app[4]
    if price == '0.0':
        ios_final.append(app)
        
print(len(android_final))
print(len(ios_final))

8865
4056


In [80]:
explore_data(android_final, 0, 2)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']




### Most successfull apps by genre

Our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.
Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

In [81]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    return table_percentages


def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [83]:
display_table(android_final, 1) # Category

FAMILY : 18.905809362662154
GAME : 9.723632261703328
TOOLS : 8.460236886632826
BUSINESS : 4.591088550479413
LIFESTYLE : 3.914269599548787
PRODUCTIVITY : 3.8917089678511
FINANCE : 3.699943598420756
MEDICAL : 3.5307388606880994
SPORTS : 3.395375070501974
PERSONALIZATION : 3.3164128595600673
COMMUNICATION : 3.2374506486181613
HEALTH_AND_FITNESS : 3.0795262267343486
PHOTOGRAPHY : 2.9441624365482233
NEWS_AND_MAGAZINES : 2.7975183305132543
SOCIAL : 2.662154540327129
TRAVEL_AND_LOCAL : 2.33502538071066
SHOPPING : 2.2447828539199097
BOOKS_AND_REFERENCE : 2.143260011280316
DATING : 1.8612521150592216
VIDEO_PLAYERS : 1.793570219966159
MAPS_AND_NAVIGATION : 1.3987591652566271
FOOD_AND_DRINK : 1.2408347433728144
EDUCATION : 1.161872532430908
ENTERTAINMENT : 0.9588268471517203
LIBRARIES_AND_DEMO : 0.9362662154540328
AUTO_AND_VEHICLES : 0.924985899605189
HOUSE_AND_HOME : 0.8234630569655951
WEATHER : 0.8009024252679076
EVENTS : 0.7106598984771574
PARENTING : 0.6542583192329385
ART_AND_DESIGN : 0.6429

From the category percentages table we can see that 'FAMILY' is the pre-dominant category with a percentage of 18%. Upon further investigation on the PlayStore, I found that this category is mostly games for kids. Alongwith the 'FAMILY' category we can see that the Play Store is dominated by utility apps meant for Education, Business, Productivity, Lifestyle finance.

In [86]:
# further inspecting the genres column of the PlayStore dataset
display_table(android_final, -4)

Tools : 8.44895657078398
Entertainment : 6.068809926677947
Education : 5.346869712351946
Business : 4.591088550479413
Productivity : 3.8917089678511
Lifestyle : 3.8917089678511
Finance : 3.699943598420756
Medical : 3.5307388606880994
Sports : 3.4630569655950363
Personalization : 3.3164128595600673
Communication : 3.2374506486181613
Action : 3.102086858432036
Health & Fitness : 3.0795262267343486
Photography : 2.9441624365482233
News & Magazines : 2.7975183305132543
Social : 2.662154540327129
Travel & Local : 2.323745064861816
Shopping : 2.2447828539199097
Books & Reference : 2.143260011280316
Simulation : 2.0417371686407217
Dating : 1.8612521150592216
Arcade : 1.849971799210378
Video Players & Editors : 1.7710095882684715
Casual : 1.7597292724196276
Maps & Navigation : 1.3987591652566271
Food & Drink : 1.2408347433728144
Puzzle : 1.1280315848843767
Racing : 0.9926677946982515
Role Playing : 0.9362662154540328
Libraries & Demo : 0.9362662154540328
Auto & Vehicles : 0.924985899605189
Str

Genres columns from the PlasyStore is quite granular with a lot of categories. But here also we see the same
pattern that tool and utility apps are dominate the market. Of course, this does not imply more number of users.

In [48]:
display_table(ios_final, 11)

Games : 55.64595660749507
Entertainment : 8.234714003944774
Photo & Video : 4.117357001972387
Social Networking : 3.5256410256410255
Education : 3.2544378698224854
Shopping : 2.983234714003945
Utilities : 2.687376725838264
Lifestyle : 2.3175542406311638
Finance : 2.0710059171597637
Sports : 1.947731755424063
Health & Fitness : 1.8737672583826428
Music : 1.6518737672583828
Book : 1.6272189349112427
Productivity : 1.5285996055226825
News : 1.4299802761341223
Travel : 1.3806706114398422
Food & Drink : 1.0601577909270217
Weather : 0.7642998027613412
Reference : 0.4930966469428008
Navigation : 0.4930966469428008
Business : 0.4930966469428008
Catalogs : 0.22189349112426035
Medical : 0.19723865877712032


The landscape of ios apps seems completely different compared to PlayStore. Market is very heavily dominated by
games apps(close to 56%). However, this does not imply that greatest number of users i.e. demand might not be the same 
as the offer.

### Popular app by Genre on Google Play Store

For PlayStore apps dataset we have the number of installs, so we can get clear picture of genre popularity from that column information

In [88]:
display_table(android_final, 5)

1,000,000+ : 15.724760293288211
100,000+ : 11.551043429216017
10,000,000+ : 10.547095318668923
10,000+ : 10.197405527354766
1,000+ : 8.403835307388608
100+ : 6.91483361534123
5,000,000+ : 6.824591088550479
500,000+ : 5.561195713479977
50,000+ : 4.771573604060913
5,000+ : 4.512126339537507
10+ : 3.542019176536943
500+ : 3.248730964467005
50,000,000+ : 2.3011844331641287
100,000,000+ : 2.131979695431472
50+ : 1.9176536943034406
5+ : 0.7896221094190639
1+ : 0.5076142131979695
500,000,000+ : 0.2707275803722504
1,000,000,000+ : 0.2256063169768754
0+ : 0.04512126339537507
0 : 0.011280315848843767


We can see that the installs count is not an absolute number but a string which gives us a rough idea like 50+ or 1000+. As we are looking to analyze popularity by CATEGORY we do not need exact numbers so this estimate is good for our analysis. We process this strings to remove ',' and '+' characters.

In [91]:
category_android_ft = freq_table(android_final, 5)


{'10,000+': 10.197405527354766,
 '5,000,000+': 6.824591088550479,
 '50,000,000+': 2.3011844331641287,
 '100,000+': 11.551043429216017,
 '50,000+': 4.771573604060913,
 '1,000,000+': 15.724760293288211,
 '10,000,000+': 10.547095318668923,
 '5,000+': 4.512126339537507,
 '500,000+': 5.561195713479977,
 '1,000,000,000+': 0.2256063169768754,
 '100,000,000+': 2.131979695431472,
 '1,000+': 8.403835307388608,
 '500,000,000+': 0.2707275803722504,
 '500+': 3.248730964467005,
 '100+': 6.91483361534123,
 '50+': 1.9176536943034406,
 '10+': 3.542019176536943,
 '1+': 0.5076142131979695,
 '5+': 0.7896221094190639,
 '0+': 0.04512126339537507,
 '0': 0.011280315848843767}